In [9]:
pTableName = 'table1'
pJoinKey= 'salekey'
pOrderKey=''

StatementMeta(, ea568abd-cea4-4d6a-b05b-d4a5abc7842b, 11, Finished, Available)

In [10]:
from pyspark.sql.types import *
from pyspark.sql.functions import col,lit,current_timestamp

spark.sql("set spark.sql.streaming.schemaInference=true")

relbaselocation = "Files/AutoMerger"

def loadIncrementals(table_name, join_key, order_key):

    def upsertToDeltaCaptureCDC(microBatchOutputDF, batchId):
        targettable = microBatchOutputDF.select("fep_table_name").limit(1).collect()[0][0]
        joinkey = microBatchOutputDF.select("fep_join_key").limit(1).collect()[0][0]
        orderkey = microBatchOutputDF.select("fep_order_key").limit(1).collect()[0][0]
        #add this to the end of the next line if you require a timestmape field .withColumn("changeTimestamp",current_timestamp())
        microBatchOutputDF = microBatchOutputDF.drop("fep_join_key").drop('fep_table_name')
        microBatchOutputDF.createOrReplaceTempView("updates")
        tablecols = spark.sql('show columns in table0').collect()
        updatestmt = ''
        insertsrcstmt = ''
        inserttgtstmt = ''
        for column in tablecols:
          #print(column.col_name)
          updatestmt = updatestmt+column.col_name +'= source.'+column.col_name + ','
          insertsrcstmt = insertsrcstmt + column.col_name+ ','
          inserttgtstmt = inserttgtstmt + "source."+column.col_name+ ','
        #updatestmt = updatestmt + 'changeTimestamp = source.changeTimestamp' 
        updatestmt = updatestmt.rstrip(",")
        insertsrcstmt = insertsrcstmt.rstrip(",")
        inserttgtstmt = inserttgtstmt.rstrip(",")
        #insertsrcstmt =  insertsrcstmt + 'changeTimestamp'
        #inserttgtstmt = inserttgtstmt + 'source.changeTimestamp'
        if orderkey != '':
          mergestmt = '''MERGE INTO '''+targettable+''' target
          USING (SELECT X.* FROM
                (
                SELECT
                    DENSE_RANK() OVER (PARTITION BY CT.'''+joinkey+''' ORDER BY CT.'''+orderkey+''' desc) AS DR
                ,   CT.*
                FROM
                    updates AS CT
                ) X
                where DR=1
                )  source 
          ON source.'''+ joinkey + '''=  target.'''+joinkey+'''
          WHEN MATCHED THEN UPDATE SET ''' + updatestmt + '''
          WHEN NOT MATCHED THEN INSERT (''' + insertsrcstmt + ''') VALUES ('''+inserttgtstmt + ''')'''
        else:
          mergestmt = '''MERGE INTO '''+targettable+''' target
          USING updates AS source 
          ON source.'''+ joinkey + '''=  target.'''+joinkey+'''
          WHEN MATCHED THEN UPDATE SET ''' + updatestmt + '''
          WHEN NOT MATCHED THEN INSERT (''' + insertsrcstmt + ''') VALUES ('''+inserttgtstmt + ''')'''
        #display(mergestmt)
        microBatchOutputDF.sparkSession.sql(mergestmt)
  

    streamingUpserts = spark.readStream.format("parquet").load(relbaselocation+"/incrementalfeed/"+table_name+"/").withColumn("fep_table_name", lit(table_name)) \
      .withColumn("fep_join_key", lit(join_key)).withColumn("fep_order_key", lit(order_key)) 
    #display(streamingUpserts)

    #.trigger(once=True) \
    #Apply to the main-delta-table
    #trigger(once=True).
    query = streamingUpserts.writeStream.queryName(table_name).format("delta").foreachBatch(upsertToDeltaCaptureCDC).option("checkpointLocation","Files/checkpoints/"+table_name+"/").outputMode("update").start()
    try:
      query.awaitTermination()
    except:
      #query.stop(stopSparkContext = true, stopGracefully = true)
      mssparkutils.notebook.exit("Error loading "+ pTableName + ":" +  sys.exc_info()[0])
      raise
    
loadIncrementals(pTableName,pJoinKey,pOrderKey)

StatementMeta(, ea568abd-cea4-4d6a-b05b-d4a5abc7842b, 12, Finished, Available)

In [11]:
mssparkutils.notebook.exit("Successfully loaded "+ pTableName)

StatementMeta(, ea568abd-cea4-4d6a-b05b-d4a5abc7842b, 13, Finished, Available)

ExitValue: Successfully loaded table11